In [1]:
import numpy as np
from torch.utils import data
from torch.utils.data import Dataset
from torch.utils.data.sampler import SubsetRandomSampler
from sklearn import preprocessing
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import TensorDataset, DataLoader
from torch.optim.lr_scheduler import _LRScheduler
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score

class Dataset(data.Dataset):
    def __init__(self, X1, Y1):
        self.X1 = X1
        self.Y1 = Y1

    def __len__(self):
        return len(self.X1)

    def __getitem__(self, index):
        x = self.X1[index]
        y1 = self.Y1[index]
        return x, y1
    
seed = 2021
torch.manual_seed(seed)
torch.cuda.manual_seed(0)
np.random.seed(seed)
torch.backends.cudnn.deterministic = True

def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    numpy.random.seed(worker_seed)
    random.seed(worker_seed)

xTrain=np.load("../extracted_features/0/GRP-3/input_features_train_grp3.npy")
yTrain=np.load("../extracted_features/0/GRP-3/labels_train_grp3.npy")

xTest=np.load("../extracted_features/0/GRP-3/input_features_test_grp3.npy")
yTest=np.load("../extracted_features/0/GRP-3/labels_test_grp3.npy")


xVal=np.load("../extracted_features/0/GRP-3/input_features_val_grp3.npy")
yVal=np.load("../extracted_features/0/GRP-3/labels_val_grp3.npy")

train_features=np.load("../extracted_features/bl/0/GRP-3/input_features_train_grp3.npy")
labels_train=np.load("../extracted_features/bl/0/GRP-3/labels_train_grp3.npy")

test_features=np.load("../extracted_features/bl/0/GRP-3/input_features_test_grp3.npy")
labels_test=np.load("../extracted_features/bl/0/GRP-3/labels_test_grp3.npy")

val_features=np.load("../extracted_features/bl/0/GRP-3/input_features_val_grp3.npy")
labels_val=np.load("../extracted_features/bl/0/GRP-3/labels_val_grp3.npy")



In [2]:
print(labels_train.shape,labels_train)

(497,) [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1

In [3]:
print(train_features.shape,labels_train.shape)
print(test_features.shape,labels_test.shape)
print(val_features.shape,labels_val.shape)

(497, 19, 24) (497,)
(119, 19, 24) (119,)
(110, 19, 24) (110,)


In [4]:
print(xTrain.shape,yTrain.shape)
print(xTest.shape,yTest.shape)
print(xVal.shape,yVal.shape)

(512, 19, 24) (512,)
(122, 19, 24) (122,)
(118, 19, 24) (118,)


In [5]:
# xTrain=np.append(train_features,xTrain,axis=0)
# xTest=np.append(test_features,xTest,axis=0)
# xVal=np.append(val_features,xVal,axis=0)
# yTrain=np.append(labels_train,yTrain,axis=0)
# yTest=np.append(labels_test,yTest,axis=0)
# yVal=np.append(labels_val,yVal,axis=0)

In [6]:
import os

def readFeatures(path):
    features=np.array([])
    start=0
    for file in os.listdir(path):
        d = os.path.join(path, file)
        datafile=d+"/features.npy"
        try:
            temp=np.load(datafile)
#             print(datafile)
#             print(temp.shape)
            if(start==0):
                features=temp
                if(d[-10]=='S'):
                    label=np.ones(len(temp))
                else:
                    label=np.zeros(len(temp))
                start=1
            else:
                features=np.vstack((features,temp))
                if(d[-10]=='S'):
                    label=np.append(label,np.ones(len(temp)))
                else:
                    label=np.append(label,np.zeros(len(temp)))
        except IOError:
            print('file not in Scripted')

            
    return features,label

def load_data(path,mode):
    if mode == "test":
        xTest,yTest=readFeatures(path+"/"+mode)
#         xTest = xTest.reshape(len(xTest),1,19,24)
        return Dataset(xTest,yTest)
        
    elif mode == "train":
        xTrain,yTrain=readFeatures(path+"/"+mode)
#         xTrain = xTrain.reshape(len(xTrain),1,19,24)
        return Dataset(xTrain,yTrain)
        
    elif mode == "val":
        xVal,yVal=readFeatures(path+"/"+mode)
#         xVal = xVal.reshape(len(xVal),1,19,24)
        return Dataset(xVal,yVal)
    else:
        print("Mode not defined")
        return

In [7]:
batch_size=20

# train_data = Dataset(xTrain, yTrain)
# test_data=Dataset(xTest,yTest)
# val_data=Dataset(xVal,yVal)

train_data = load_data("../Participant_wise",'train')
test_data=load_data("../Participant_wise",'test')
val_data=load_data("../Participant_wise",'val')

train_data_loader = DataLoader(train_data,worker_init_fn=seed_worker, shuffle=True, batch_size=batch_size)
val_data_loader=DataLoader(val_data,worker_init_fn=seed_worker,shuffle=True,batch_size=batch_size)
test_data_loader=DataLoader(test_data,worker_init_fn=seed_worker,shuffle=True,batch_size=batch_size)

In [8]:
class CyclicLR(_LRScheduler):
    
    def __init__(self, optimizer, schedule, last_epoch=-1):
        assert callable(schedule)
        self.schedule = schedule
        super().__init__(optimizer, last_epoch)

    def get_lr(self):
        return [self.schedule(self.last_epoch, lr) for lr in self.base_lrs]

In [9]:
def cosine(t_max, eta_min=0):
    
    def scheduler(epoch, base_lr):
        t = epoch % t_max
        return eta_min + (base_lr - eta_min)*(1 + np.cos(np.pi*t/t_max))/2
    
    return scheduler



In [10]:
class Classifier(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.input_dim = 24
        self.hidden_dim = 500
        self.L=250
        self.layer_dim = 4
        self.output_dim=1
        self.fc1 = nn.Sequential(
            nn.Linear(self.hidden_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )
        
        self.rnn = nn.LSTM(self.input_dim, self.hidden_dim, self.layer_dim, batch_first=True)
        
        self.attention = nn.Sequential(
            nn.Linear(self.hidden_dim, self.L),
            nn.Tanh(),
            nn.Linear(self.L, 1)
        )
    
    def forward(self, x):
        h0, c0 = self.init_hidden(x)
        out, (hn, cn) = self.rnn(x, (h0, c0))

        A = self.attention(out) 
        A = torch.transpose(A, 2,1)  
        A = F.softmax(A, dim=2) 
#         print(A.shape)
        M = torch.matmul(A, out) .squeeze(1)
#         print(M.shape)
        out=self.fc1(M)
#         print(out.shape)
        return out
    
    def init_hidden(self, x):
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim)
        c0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim)
        return [t.cuda() for t in (h0, c0)]

In [11]:
def train(model,train_data_loader):
    best_acc1 = 0
    modelname=[]
    truth=[]
    preds=[]
    model.train()
    total=len(train_data_loader)*batch_size
    train_loss = 0.
    for minibatch in train_data_loader:
        X, Y1  = minibatch
        X=X.cuda()
        Y1=Y1.cuda()
        output = model(X.float())
        output=output.squeeze(1)
        loss = loss_func(output, Y1.float())
        Y_hat1 = torch.ge(output, 0.5).float()
        train_loss += loss.item()
        truth.extend(Y1.tolist())
        preds.extend(Y_hat1.tolist())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        sched.step()
    trainacc1=accuracy_score(truth,preds)
    train_loss /= total
    print("EPOCH ",it)
    print('Train : Loss: {:.4f}, Train acc1 : {:.4f}'.format(train_loss,trainacc1))

def evalue(model,best_acc1,val_data_loader,modelname,it):
    count=0
    val_loss= 0.
    truth=[]
    preds=[]
    total=len(val_data_loader)*batch_size
    model.eval()
    best_model="./saved_models/best_model"
    for minibatch in val_data_loader:
        X_valid, Y1_valid  = minibatch
        X_valid=X_valid.cuda()
        Y1_valid=Y1_valid.cuda()
        output_val = model(X_valid.float())
        output_val=output_val.squeeze(1)
        loss = loss_func(output_val, Y1_valid.float())
        Y_hat1_val = torch.ge(output_val, 0.5).float()
        val_loss += loss.item()
        truth.extend(Y1_valid.tolist())
        preds.extend(Y_hat1_val.tolist())
    valacc1=accuracy_score(truth,preds)
    val_loss /= total
    print('Val : Loss: {:.4f}, Val acc1 : {:.4f}'.format(val_loss,valacc1))
    if valacc1 >= best_acc1:
        best_acc1 = valacc1
        best_state = model.state_dict()
        torch.save(best_state, best_model+'_epoch_'+str(it)+".pth")
        modelname.append(best_model+'_epoch_'+str(it)+".pth")
        print('Best validation accuracy1 ', best_acc1)
    return best_acc1
    

In [12]:
feature_size = 24
shared_layer_size = 512
LR = 0.0001
epoch = 200
model=Classifier()
model.cuda()
iterations_per_epoch = len(train_data_loader)
loss_func = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)
sched = CyclicLR(optimizer, cosine(t_max=iterations_per_epoch * 2, eta_min=LR/100))

In [13]:
best_acc1=0
modelname=[]
for it in range(epoch+1):
    train(model,train_data_loader)
    best_acc1=evalue(model,best_acc1,val_data_loader,modelname,it)


EPOCH  0
Train : Loss: 0.0346, Train acc1 : 0.5504
Val : Loss: 0.0345, Val acc1 : 0.5085
Best validation accuracy1  0.5084745762711864
EPOCH  1
Train : Loss: 0.0344, Train acc1 : 0.5446
Val : Loss: 0.0345, Val acc1 : 0.5169
Best validation accuracy1  0.5169491525423728
EPOCH  2
Train : Loss: 0.0340, Train acc1 : 0.5756
Val : Loss: 0.0352, Val acc1 : 0.6610
Best validation accuracy1  0.6610169491525424
EPOCH  3
Train : Loss: 0.0333, Train acc1 : 0.5891
Val : Loss: 0.0370, Val acc1 : 0.6780
Best validation accuracy1  0.6779661016949152
EPOCH  4
Train : Loss: 0.0325, Train acc1 : 0.6260
Val : Loss: 0.0355, Val acc1 : 0.5424
EPOCH  5
Train : Loss: 0.0320, Train acc1 : 0.6124
Val : Loss: 0.0372, Val acc1 : 0.6610
EPOCH  6
Train : Loss: 0.0316, Train acc1 : 0.6686
Val : Loss: 0.0376, Val acc1 : 0.5763
EPOCH  7
Train : Loss: 0.0297, Train acc1 : 0.6550
Val : Loss: 0.0367, Val acc1 : 0.5254
EPOCH  8
Train : Loss: 0.0298, Train acc1 : 0.6550
Val : Loss: 0.0297, Val acc1 : 0.6949
Best validation

EPOCH  89
Train : Loss: 0.0048, Train acc1 : 0.9612
Val : Loss: 0.1037, Val acc1 : 0.5932
EPOCH  90
Train : Loss: 0.0046, Train acc1 : 0.9612
Val : Loss: 0.1235, Val acc1 : 0.5508
EPOCH  91
Train : Loss: 0.0030, Train acc1 : 0.9845
Val : Loss: 0.1233, Val acc1 : 0.5678
EPOCH  92
Train : Loss: 0.0045, Train acc1 : 0.9593
Val : Loss: 0.1060, Val acc1 : 0.6102
EPOCH  93
Train : Loss: 0.0040, Train acc1 : 0.9651
Val : Loss: 0.1114, Val acc1 : 0.6017
EPOCH  94
Train : Loss: 0.0028, Train acc1 : 0.9845
Val : Loss: 0.1328, Val acc1 : 0.5763
EPOCH  95
Train : Loss: 0.0012, Train acc1 : 0.9942
Val : Loss: 0.1405, Val acc1 : 0.5593
EPOCH  96
Train : Loss: 0.0014, Train acc1 : 0.9942
Val : Loss: 0.1620, Val acc1 : 0.5424
EPOCH  97
Train : Loss: 0.0008, Train acc1 : 0.9922
Val : Loss: 0.1606, Val acc1 : 0.5424
EPOCH  98
Train : Loss: 0.0023, Train acc1 : 0.9826
Val : Loss: 0.1403, Val acc1 : 0.5678
EPOCH  99
Train : Loss: 0.0025, Train acc1 : 0.9826
Val : Loss: 0.1612, Val acc1 : 0.5254
EPOCH  100

EPOCH  180
Train : Loss: 0.0015, Train acc1 : 0.9884
Val : Loss: 0.1453, Val acc1 : 0.5932
EPOCH  181
Train : Loss: 0.0007, Train acc1 : 0.9942
Val : Loss: 0.1523, Val acc1 : 0.5763
EPOCH  182
Train : Loss: 0.0137, Train acc1 : 0.9128
Val : Loss: 0.0700, Val acc1 : 0.5847
EPOCH  183
Train : Loss: 0.0141, Train acc1 : 0.8469
Val : Loss: 0.0698, Val acc1 : 0.5593
EPOCH  184
Train : Loss: 0.0087, Train acc1 : 0.9341
Val : Loss: 0.1028, Val acc1 : 0.5508
EPOCH  185
Train : Loss: 0.0058, Train acc1 : 0.9399
Val : Loss: 0.1218, Val acc1 : 0.5169
EPOCH  186
Train : Loss: 0.0053, Train acc1 : 0.9496
Val : Loss: 0.1293, Val acc1 : 0.5254
EPOCH  187
Train : Loss: 0.0040, Train acc1 : 0.9554
Val : Loss: 0.1372, Val acc1 : 0.5339
EPOCH  188
Train : Loss: 0.0037, Train acc1 : 0.9593
Val : Loss: 0.1360, Val acc1 : 0.5000
EPOCH  189
Train : Loss: 0.0032, Train acc1 : 0.9651
Val : Loss: 0.1515, Val acc1 : 0.4915
EPOCH  190
Train : Loss: 0.0030, Train acc1 : 0.9632
Val : Loss: 0.1643, Val acc1 : 0.4915

In [14]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score

modeltest=Classifier()
best_state=torch.load(modelname[-1])
modeltest.load_state_dict(best_state)
modeltest.cuda()
modeltest.eval()
truth=[]
preds=[]
for minibatch in test_data_loader:
            X_test, Y1_test  = minibatch
            X_test=X_test.cuda()
            Y1_test=Y1_test.cuda()
            output_test = modeltest(X_test.float())
            output_test=output_test.squeeze(1)
            prediction = torch.ge(output_test, 0.5).float()
            truth.extend(Y1_test.tolist())
            preds.extend(prediction.tolist())
acc=accuracy_score(truth,preds)
# print(truth,preds)
tn, fp, fn, tp = confusion_matrix(truth, preds).ravel()
f1score=f1_score(truth, preds)
precision=precision_score(truth, preds)
recall=recall_score(truth,preds)
roc=roc_auc_score(truth,preds)
specificity=tn/(tn+fp)

print('{:.2f} {:.2f} {:.2f} {:.2f} {:.2f} {:.2f}'.format(acc,f1score,precision,recall,roc,specificity))


0.74 0.71 0.78 0.66 0.74 0.82


In [15]:
testdataset = load_data("../Participant_wise_win-1",'test')

test_data_loader = DataLoader(testdataset,worker_init_fn=seed_worker,shuffle=True,batch_size=batch_size)

modeltest=Classifier()
best_state=torch.load(modelname[-1])
modeltest.load_state_dict(best_state)
modeltest.cuda()
modeltest.eval()
truth=[]
preds=[]
for minibatch in test_data_loader:
            X_test, Y1_test  = minibatch
            X_test=X_test.cuda()
            Y1_test=Y1_test.cuda()
            output_test = modeltest(X_test.float())
            output_test=output_test.squeeze(1)
            prediction = torch.ge(output_test, 0.5).float()
            truth.extend(Y1_test.tolist())
            preds.extend(prediction.tolist())
acc=accuracy_score(truth,preds)
# print(truth,preds)
tn, fp, fn, tp = confusion_matrix(truth, preds).ravel()
f1score=f1_score(truth, preds)
precision=precision_score(truth, preds)
recall=recall_score(truth,preds)
roc=roc_auc_score(truth,preds)
specificity=tn/(tn+fp)

print('{:.2f} {:.2f} {:.2f} {:.2f} {:.2f} {:.2f}'.format(acc,f1score,precision,recall,roc,specificity))


0.69 0.65 0.74 0.59 0.69 0.80


In [16]:
testdataset = load_data("../Participant_wise_win-2",'test')

test_data_loader = DataLoader(testdataset,worker_init_fn=seed_worker,shuffle=True,batch_size=batch_size)

modeltest=Classifier()
best_state=torch.load(modelname[-1])
modeltest.load_state_dict(best_state)
modeltest.cuda()
modeltest.eval()
truth=[]
preds=[]
for minibatch in test_data_loader:
            X_test, Y1_test  = minibatch
            X_test=X_test.cuda()
            Y1_test=Y1_test.cuda()
            output_test = modeltest(X_test.float())
            output_test=output_test.squeeze(1)
            prediction = torch.ge(output_test, 0.5).float()
            truth.extend(Y1_test.tolist())
            preds.extend(prediction.tolist())
acc=accuracy_score(truth,preds)
# print(truth,preds)
tn, fp, fn, tp = confusion_matrix(truth, preds).ravel()
f1score=f1_score(truth, preds)
precision=precision_score(truth, preds)
recall=recall_score(truth,preds)
roc=roc_auc_score(truth,preds)
specificity=tn/(tn+fp)

print('{:.2f} {:.2f} {:.2f} {:.2f} {:.2f} {:.2f}'.format(acc,f1score,precision,recall,roc,specificity))


0.78 0.76 0.81 0.72 0.78 0.83


In [17]:
# for i in range(len(modelname)):
#     modeltest=Classifier()
#     best_state=torch.load(modelname[i])
#     modeltest.load_state_dict(best_state)
#     modeltest.cuda()
#     modeltest.eval()
#     truth=[]
#     preds=[]
#     for minibatch in test_data_loader:
#                 X_test, Y1_test  = minibatch
#                 X_test=X_test.cuda()
#                 Y1_test=Y1_test.cuda()
#                 output_test = modeltest(X_test.float())
#                 output_test=output_test.squeeze(1)
#                 prediction = torch.ge(output_test, 0.5).float()
#                 truth.extend(Y1_test.tolist())
#                 preds.extend(prediction.tolist())
#     acc=accuracy_score(truth,preds)
#     # print(truth,preds)
#     tn, fp, fn, tp = confusion_matrix(truth, preds).ravel()
#     f1score=f1_score(truth, preds)
#     precision=precision_score(truth, preds)
#     recall=recall_score(truth,preds)
#     roc=roc_auc_score(truth,preds)
#     specificity=tn/(tn+fp)

#     print('{:.2f} {:.2f} {:.2f} {:.2f} {:.2f} {:.2f}'.format(acc,f1score,precision,recall,roc,specificity))

0.51 0.00 0.00 0.00 0.50 1.00
0.51 0.00 0.00 0.00 0.50 1.00


/home/hsharm04/.conda/envs/physio/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/hsharm04/.conda/envs/physio/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.56 0.35 0.64 0.24 0.55 0.87
0.51 0.06 0.50 0.03 0.50 0.97
0.41 0.29 0.35 0.24 0.40 0.57
0.78 0.76 0.81 0.72 0.78 0.83
